The purpose of this file is to debug any data issues occurring on the Coresignal data and its processing.


In [6]:
import pandas as pd
import os
import sys

os.chdir('/shared/share_scp/coresignal/gitrepo_facebook')
if '/shared/share_scp/coresignal/gitrepo_facebook' in sys.modules:
    for module_name in list(sys.modules.keys()):
        if module_name.startswith('university_name_matcher'):
            del sys.modules[module_name]
from university_name_matcher import university_name_matcher 
import sys
import importlib

os.chdir('/shared/share_scp/coresignal')

In [ ]:
import random


chunk_size = 10_000
university_urls_keep = []
university_urls_remove = []
university_urls_to_unitid_correspondence = pd.DataFrame()
all_linkedin_matches = pd.DataFrame()


#with open(csv_path, 'r') as f:
#    header = f.readline().strip().split(',')

# Calculate rows to skip (excluding header)
skiprows = None

#skiprows = 
nrows = random.randint(0, 20_000_000)
csv_path = "coresignal_member_education_1.csv"



chunk_iter = pd.read_csv(
    csv_path,
    header=0,
    skiprows=skiprows,
    usecols=['id','activities_and_societies','description','member_id', 'title', 'subtitle','date_from', 'date_to', 'school_url'],
    dtype={'id': int, 'member_id': int, 'title': str, 'activities_and_societies': str, 'subtitle':str,'date_from': str, 'date_to': str, 'school_url': str},
    nrows=nrows,
    low_memory=False,
    iterator=True,
    chunksize=chunk_size
)

all_dups = pd.DataFrame()

for i, chunk_data in enumerate(chunk_iter):  
    data = chunk_data[~chunk_data['school_url'].isnull()]
    umatcher = university_name_matcher()

    universities_adopted_facebook = umatcher.load_university_data()
    top_10_tags = umatcher.common_tags_to_remove()
    matched = umatcher.match_by_tags_and_clean_name(universities_adopted_facebook,data)
    dup_member_ids = matched[matched.duplicated(['member_id','school_url'])].member_id.unique()

    dups = matched.loc[matched.member_id.isin(dup_member_ids),['id','clean_name_lm','clean_name_ua','member_id','title','subtitle','school_url','instnm','date_from','date_to']].sort_values('member_id')

    all_dups = pd.concat([dups, all_dups], ignore_index=True)
    print(f"Chunk {i} done, found {len(dups)} duplicates, total duplicates so far: {len(all_dups)}")

    if i > 5: 
        break


	matching: preparing data
	matching: filtering to rows with matched tags
	matching: computing levenshtein distance
	matching: time elapsed 0:00:04 (hh:mm:ss)
	matching: filtering based on cutoff
	matching: getting the best matches
Chunk 0 done, found 24 duplicates, total duplicates so far: 24
	matching: preparing data
	matching: filtering to rows with matched tags
	matching: computing levenshtein distance
	matching: time elapsed 0:00:04 (hh:mm:ss)
	matching: filtering based on cutoff
	matching: getting the best matches
Chunk 1 done, found 23 duplicates, total duplicates so far: 47
	matching: preparing data
	matching: filtering to rows with matched tags
	matching: computing levenshtein distance


KeyboardInterrupt: 

In [18]:

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(all_dups.sort_values(['member_id','date_from','date_to']))


,id,clean_name_lm,clean_name_ua,member_id,title,subtitle,school_url,instnm,date_from,date_to
1,9763911,snead,snead,18632336,Snead State Community College,Associate of Science (A.S.),None,Snead State Community College,2009,2011
0,9763910,virginia -birmingham,virginia -birmingham,18632336,Virginia College-Birmingham,"Associate's degree, Registered Nursing/Registe...",None,Virginia College-Birmingham,2011,2013
2,9764109,coastal bend,coastal bend,18632505,Coastal Bend College,"Associate of Arts and Sciences (A.A.S.), Appli...",None,Coastal Bend College,1989,1991
3,9764108,the of texas health science center at san antonio,the of texas health science center at san antonio,18632505,The University of Texas Health Science Center ...,"Bachelor of Science (BS), Clinical Laboratory ...",None,The University of Texas Health Science Center ...,1992,1994
24,51800581,suffolk county,suffolk county,85463837,Suffolk County Community College,"Associate of Arts (A.A.), Liberal Arts and Sci...",None,Suffolk County Community College,1990,1994
23,51800580,daytona,daytona,85463837,Daytona State College,"Bachelor of Applied Science (B.A.Sc.), Supervi...",None,Daytona State College,2015,2016
25,51802352,mount san antonio,mt san antonio,85466378,Mount San Antonio College,"Associate Degree, Air Transportation",None,Mt San Antonio College,1984,1988
27,284018193,mt. san antonio,mt san antonio,85466378,Mt. San Antonio College,"AS, Commercial Flight",https://www.linkedin.com/edu/mt%2E-san-antonio...,Mt San Antonio College,1984,1989
26,51802350,"claifornia , los angeles",california -los angeles,85466378,"Claifornia State University, Los Angeles","BS, Industrial Technology W/ Option Aviation A...",None,California State University-Los Angeles,1988,1992
28,51802620,washington of professional psychology,wisconsin of professional psychology,85466742,Washington School of Professional Psychology,"M.A., Clinical Psychology",None,Wisconsin School of Professional Psychology,2000,None


,id,member_id,title,subtitle,school_url,instnm,date_from,date_to
965,9305233,18022583,Douglas J Aveda Institute,", Cosmetology and Related Personal Grooming Se...",https://www.linkedin.com/edu/douglas-j-aveda-i...,Douglas J Aveda Institute,2012,2013
4151,9305235,18022583,Virginia Farrell Beauty School-Ferndale,", Cosmetology/Cosmetologist, General",None,Virginia Farrell Beauty School-Ferndale,1999,2001
275,9763856,18632302,Barton County Community College,"Associate's Degree, Medical Laboratory Technician",https://es.linkedin.com/edu/barton-county-comm...,Barton County Community College,1997,2009
408,9763857,18632302,Brown Mackie College-Salina,"Certificate in Networking, Information Technology",https://es.linkedin.com/edu/brown-mackie-colle...,Brown Mackie College-Salina,2003,2004
3518,9763911,18632336,Snead State Community College,Associate of Science (A.S.),https://us.linkedin.com/edu/snead-state-commun...,Snead State Community College,2009,2011
...,...,...,...,...,...,...,...,...
1008,507255160,388727245,Eastern Kentucky University,"Master of Science (MS), Biology/Biological Sci...",https://nz.linkedin.com/edu/eastern-kentucky-u...,Eastern Kentucky University,1994,1996
1302,318493377,391980172,Graham Hospital School of Nursing,"Diploma, Registered Nursing/Registered Nurse",https://www.linkedin.com/edu/graham-hospital-s...,Graham Hospital School of Nursing,2010,2013
3637,318493378,391980172,Spoon River College,"Associate's degree, Physical Sciences",https://www.linkedin.com/edu/spoon-river-colle...,Spoon River College,2012,None
4101,318494719,391981342,UT El Paso,"BS, MA, Biology, Medicine, Psychology",None,CET-El Paso,1968,1986
